### Importação de Biblioteca

In [1]:
import os, random
random.seed(2)
import numpy as np
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromFile

import tensorflow as tf
from sklearn.utils import resample
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import time

import math

from IPython.core.display import HTML
from pretty_confusion_matrix import pp_matrix_from_data

### Modificando design do jupyter Notebook

In [ ]:
def jupyter_settings():
    display(HTML('<style>.container {width:100% !important;}</style>'))
jupyter_settings()

# ------------------------------------------------
# --------(Criando Chips de Imagem)--------
# ------------------------------------------------

### Váriaveis - Criando Chips de Imagem

In [ ]:
chipSize = 3
epochsNumber = 30
trainedModelDir = 'trained_models_' + str(chipSize) + 'by' + str(chipSize) + '_' + str(epochsNumber) + 'Epochs'
featuresName = 'CNN_' + str(chipSize) + 'by' + str(chipSize) + '_features.npy'
labelsName = 'CNN_' + str(chipSize) + 'by' + str(chipSize) + '_labels.npy'

### Change the working directory

In [ ]:
#cria diretório do treinamento
if not os.path.exists(os.path.join(os.getcwd(), trainedModelDir)):
    os.mkdir(os.path.join(os.getcwd(), trainedModelDir))
os.chdir(trainedModelDir)

### define the file names

In [ ]:
feature_file = r"/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data/ImagemVIG.tif"
label_file = r"/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data/ImagemRotulada_qgis.tif"

### create feature chips using pyrsgis

In [ ]:
features = imageChipsFromFile(feature_file, x_size=chipSize, y_size=chipSize)
print(feature_file)


In [ ]:
features

In [ ]:
features = np.rollaxis(features, 3, 1)

In [ ]:
features

### read the label file and reshape it

In [ ]:
ds, labels = raster.read(label_file)


In [ ]:
labels = labels.flatten()

In [ ]:
labels

### print basic details

In [ ]:
print('Input features shape:', features.shape)
print('\nInput labels shape:', labels.shape)
print('Values in input features, min: %d & max: %d' % (features.min(), features.max()))
print('Values in input labels, min: %d & max: %d' % (labels.min(), labels.max()))

### Save the arrays as .npy files

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), trainedModelDir)):
    os.mkdir(os.path.join(os.getcwd(), trainedModelDir))

np.save(featuresName, features)
np.save(labelsName, labels)

# ------------------------------------------------
# -------------(Treinando Modelo)------------
# ------------------------------------------------

### Variáveis - Treinando Modelo

In [2]:
chipSize = 19
epochsNumber = 30
batchsize = 512
featuresName = 'CNN_' + str(chipSize) + 'by' + str(chipSize) + '_features.npy'
labelsName = 'CNN_' + str(chipSize) + 'by' + str(chipSize) + '_labels.npy'
trainedModelDir = 'trained_models_' + str(chipSize) + 'by' + str(chipSize) + '_' + str(epochsNumber) + 'Epochs'
modelFile = trainedModelDir + '/200409_CNN_Builtup_' + str(chipSize)+ 'by' + str(chipSize) +'_' + str(epochsNumber) + 'Epochs_CNNScore{}_CNNErro{}.h5'


### Change the working directory

In [4]:
os.chdir('/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data')
os.chdir(trainedModelDir)

### Load arrays from .npy files

In [2]:
features = np.load(input())
labels = np.load(input())

/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data/x/trained_models_13by13_100Epochs/CNN_13by13_features.npy
/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data/x/trained_models_13by13_100Epochs/CNN_13by13_labels.npy


### Separate and balance the classes

In [3]:
built_features = features[labels==1]
built_labels = labels[labels==1]

unbuilt_features = features[labels==0]
unbuilt_labels = labels[labels==0]

print('Number of records in each class:')
print('Built: %d, Unbuilt: %d' % (built_labels.shape[0], unbuilt_labels.shape[0]))

Number of records in each class:
Built: 233449, Unbuilt: 3321389


### Downsample the majority class

In [4]:
unbuilt_features = resample(unbuilt_features,
                            replace = False, # sample without replacement
                            n_samples = built_features.shape[0], # match minority n
                            random_state = 2)

unbuilt_labels = resample(unbuilt_labels,
                          replace = False, # sample without replacement
                          n_samples = built_features.shape[0], # match minority n
                          random_state = 2)

print('Number of records in balanced classes:')
print('Built: %d, Unbuilt: %d' % (built_labels.shape[0], unbuilt_labels.shape[0]))

Number of records in balanced classes:
Built: 233449, Unbuilt: 233449


### Combine the balanced features

In [6]:
features = np.concatenate((built_features, unbuilt_features), axis=0)
labels = np.concatenate((built_labels, unbuilt_labels), axis=0)

### Normalise the features

In [7]:
features = features / 255.0
print('New values in input features, min: %d & max: %d' % (features.min(), features.max()))

New values in input features, min: 0 & max: 1


### Define the function to split features and labels

In [8]:
def train_test_split(features, labels, trainProp=0.7):
    dataSize = features.shape[0]
    sliceIndex = int(dataSize*trainProp)
    randIndex = np.arange(dataSize)
    random.shuffle(randIndex)
    train_x = features[[randIndex[:sliceIndex]], :, :, :][0]
    test_x = features[[randIndex[sliceIndex:]], :, :, :][0]
    train_y = labels[randIndex[:sliceIndex]]
    test_y = labels[randIndex[sliceIndex:]]
    return(train_x, train_y, test_x, test_y)

### Call the function to split the data

In [9]:
train_x, train_y, test_x, test_y = train_test_split(features, labels)

### Transpose the features to channel last format

In [10]:
train_x = tf.transpose(train_x, [0, 2, 3, 1])
test_x = tf.transpose(test_x, [0, 2, 3, 1])
print('Reshaped features:', train_x.shape, test_x.shape)
_, rowSize, colSize, nBands = train_x.shape


Metal device set to: Apple M1 Pro


2022-07-25 23:34:38.055145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 23:34:38.055569: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Reshaped features: (326828, 13, 13, 4) (140070, 13, 13, 4)


### Create a model

In [12]:
model = keras.Sequential()
model.add(Conv2D(32, kernel_size=1, padding='valid', activation='relu', input_shape=(chipSize, chipSize, nBands)))
model.add(Dropout(0.25))
model.add(Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
model.add(Dropout(0.25))
model.add(Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
model.add(Dropout(0.25))
model.add(Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer= 'rmsprop',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 19, 19, 32)        160       
                                                                 
 dropout (Dropout)           (None, 19, 19, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 48)        1584      
                                                                 
 dropout_1 (Dropout)         (None, 19, 19, 48)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 48)        2352      
                                                                 
 dropout_2 (Dropout)         (None, 19, 19, 48)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 19, 19, 48)        2

### Run the model

In [ ]:
with tf.device('/cpu:0'):
    history = model.fit(train_x, train_y, epochs=epochsNumber, batch_size=batchsize, validation_data=(test_x, test_y))

### Plota o histórico da acurácia 

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Learning curve')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.savefig('curve_accuracy.png', dpi=300)
plt.show()

### Plota o Histórico de Loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('curve_Loss.png', dpi=300)
plt.show()

### Mostra a potuação da acurácia

In [11]:
model = tf.keras.models.load_model(input())

/Users/luizramos/Repos/CNN - Deep Learning sensoreamento remoto/Data/x/trained_models_13by13_100Epochs/200409_CNN_Builtup_13by13_100Epochs_PScore0.845_RScore0.816_FScore0.830.h5


In [12]:
scores = model.evaluate(test_x, test_y, verbose=0)
result_error = str("%.2f"%(1-scores[1]))
result = str("%.2f"%(scores[1]))
print("CNN Score:", result)
print("CNN Error:", result_error)
print("CNN Loss:", str("%.2f"%(scores[0])))

2022-07-25 23:34:53.495267: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-25 23:34:53.585483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


CNN Score: 0.83
CNN Error: 0.17
CNN Loss: 0.40


In [14]:
scores

[0.39883357286453247, 0.8319054841995239]

### Salva o modelo no formato JSON

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

### Salva os pesos em HDF5

In [ ]:
model.save_weights("model_w.h5")
print("Modelo salvo no disco")

### Salva os resultados da acurácia em arquivo CSV

In [ ]:
import pandas as pd

index = []
for i in range(1, epochsNumber+1):
    index.append(f'epoca{i}')
result_train = pd.DataFrame(history.history['accuracy'], index=index)
result_test = pd.DataFrame(history.history['val_accuracy'], index=index)
result_train.to_csv('accuracy_trein.csv', header=False)
result_test.to_csv('accuracy_test.csv', header=False)

### Predict for test data 

In [ ]:
with tf.device('/gpu:0'):
    yTestPredicted = model.predict(test_x)
    yTestPredicted = yTestPredicted[:,1]

### Calculate and display the error metrics

In [ ]:
yTestPredicted = (yTestPredicted>0.6).astype(int)
cMatrix = confusion_matrix(test_y, yTestPredicted)

print("Confusion matrix:\n", cMatrix)


### Matriz confusão

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cMatrix, display_labels=['Não-coqueiro', 'coqueiro'])
disp.plot(cmap=plt.cm.Blues)
plt.savefig('nao_normalizado.png', dpi=300)
plt.show()

cMatrixNormalized = cMatrix.astype('float') / cMatrix.sum(axis=1)[:, np.newaxis]
disp2 = ConfusionMatrixDisplay(confusion_matrix=cMatrixNormalized, display_labels=['Não-coqueiro', 'coqueiro'])
disp2.plot(cmap=plt.cm.Blues)
plt.savefig('normalizado.png', dpi=300)
plt.show()

### Save the model to use later

In [ ]:
model.save(modelFile) 

# ------------------------------------------------
# ---------(Predizendo novos dados)---------
# ------------------------------------------------

### Variáveis - Predizendo novos dados

In [ ]:
chipSize = 19
new_features_dtype = 'float32'
epochsNumber = 10
#trainedModelDir = 'trained_models_' + str(chipSize) + 'by' + str(chipSize) + '_' + str(epochsNumber) + 'Epochs'
outFile = 'ImagemVIG_predicted_' + str(chipSize) + 'x' + str(chipSize) + '_' + str(epochsNumber) + 'Epochs_' + new_features_dtype + '.tif'


### Load the saved model

In [ ]:
model = tf.keras.models.load_model(input())

### Load a new multispectral image

In [ ]:
ds, featuresHyderabad = raster.read(input())

### Generate image chips in the back-end

In [ ]:
def CNNdataGenerator(mxBands, kSize):
    mxBands = mxBands / 255.0
    nBands, rows, cols = mxBands.shape
    margin = math.floor(kSize/2)
    mxBands = np.pad(mxBands, margin, mode='constant')[margin:-margin, :, :]

    features = np.empty((rows*cols, kSize, kSize, nBands), dtype=new_features_dtype)

    n = 0
    for row in range(margin, rows+margin):
        for col in range(margin, cols+margin):
            feat = mxBands[:, row-margin:row+margin+1, col-margin:col+margin+1]

            b1, b2, b3, b4 = feat
            feat = np.dstack((b1, b2, b3, b4))

            features[n, :, :, :] = feat
            n += 1
            
    return(features)

### Call the function to generate features tensor

In [ ]:
new_features = CNNdataGenerator(featuresHyderabad, kSize=chipSize)
print('Shape of the new features', new_features.shape)

### Predict new data and export the probability raster

In [ ]:
print(outFile)

In [ ]:
with tf.device('/cpu:0'):
    newPredicted = model.predict(new_features)
    newPredicted = newPredicted[:,1]
    prediction = np.reshape(newPredicted, (ds.RasterYSize, ds.RasterXSize))
    prediction = np.where(prediction < 0.6, 0, prediction)
    raster.export(prediction, ds, filename=outFile, dtype=str(new_features.dtype))